**Colab Link URL**
- https://colab.research.google.com/drive/1EbSuS0RNHzabcVvVR2SPpcT8dezgOHs0?usp=sharing

이 노트북은 [Google Colab](https://colab.research.google.com/)에서 실행하시기에 최적화되어있습니다.
상기 URL을 이용해서 코드를 실행 해보세요.

In [ ]:
import time

from openai import OpenAI, APIError, RateLimitError
from google.colab import userdata

In [ ]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("'OPENAI_API_KEY' 이름의 보안 비밀이 설정되지 않았습니다.")
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def call_openai_api(messages: list[str], temperature: float = 0.0) -> str:
    max_retries = 3
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=temperature,
            )
            return completion.choices[0].message.content
        except (APIError, RateLimitError) as e:
            print(
                f"API 오류 발생 (시도 {attempt + 1}/{max_retries}): {e}. 잠시 후 재시도합니다..."
            )
            if attempt == max_retries - 1:
                return f"오류: API 호출 실패 ({e})"
            time.sleep(2**attempt)
        except Exception as e:
            print(f"알 수 없는 오류 발생: {e}")
            return f"오류: {e}"
    return "알 수 없는 오류로 응답 생성 실패"

In [ ]:
logical_problem = """
세 명의 친구, 지아, 민준, 서연이가 각자 다른 종류의 과일 주스를 샀습니다.
주스 종류는 파인애플 주스, 바나나 주스, 딸기 주스 세 가지입니다.
다음 단서들을 이용해 누가 어떤 주스를 샀는지 알아내세요.

1. 서연이 산 주스에 사용한 과일의 한국어 이름은 민준이 산 과일 글자보다 크다.
2. 지아가 산 주스의 과일은 민준이 산 주스의 과일보다 일반적으로 크기가 작다.
3. 민준이 산 주스에 사용한 과일의 영어 이름에서 알파벳 모음 총 개수는 2보다 크다.
""".strip()

# 정답:
# 지아 딸기
# 민준 바나나
# 서연 파인애플

In [ ]:
print("--- 1. CoT가 사용되지 않은 프롬프트 방식 ---")
previous_prompt = f"""
다음 논리 문제를 풀어주세요:

문제:
{logical_problem}

아래 결과를 먼저 설명하고, 그 뒤 그 이유를 단계적으로 설명하세요.
지아: [과일 이름]
민준: [과일 이름]
서연: [과일 이름]
"""

standard_messages = [{"role": "user", "content": previous_prompt}]
result_standard = call_openai_api(standard_messages, temperature=0)
print("[CoT가 사용되지 않은 프롬프팅 결과]")
print(result_standard)
print("-" * 30)

--- 1. CoT가 사용되지 않은 프롬프트 방식 ---
[CoT가 사용되지 않은 프롬프팅 결과]
결과:
지아: 딸기 주스
민준: 파인애플 주스
서연: 바나나 주스

이유를 단계적으로 설명하겠습니다:

1. **단서 3**을 먼저 살펴보겠습니다. 민준이 산 주스에 사용한 과일의 영어 이름에서 알파벳 모음의 개수가 2보다 커야 합니다.
   - Pineapple (파인애플): 모음 4개 (i, e, a, e)
   - Banana (바나나): 모음 3개 (a, a, a)
   - Strawberry (딸기): 모음 2개 (a, e)

   따라서, 민준은 파인애플 주스나 바나나 주스를 샀을 수 있습니다.

2. **단서 1**을 살펴보겠습니다. 서연이 산 주스에 사용한 과일의 한국어 이름은 민준이 산 과일 이름보다 글자 수가 많아야 합니다.
   - 파인애플 (5글자)
   - 바나나 (3글자)
   - 딸기 (2글자)

   민준이 파인애플 주스를 샀다면, 서연은 바나나 주스를 살 수 있습니다. 민준이 바나나 주스를 샀다면, 서연은 파인애플 주스를 살 수 있습니다.

3. **단서 2**를 살펴보겠습니다. 지아가 산 주스의 과일은 민준이 산 주스의 과일보다 일반적으로 크기가 작아야 합니다.
   - 일반적으로 과일의 크기를 비교하면, 딸기 < 바나나 < 파인애플입니다.

   따라서, 민준이 파인애플 주스를 샀다면, 지아는 딸기 주스를 살 수 있습니다. 민준이 바나나 주스를 샀다면, 지아는 딸기 주스를 살 수 있습니다.

4. 모든 단서를 종합해보면:
   - 민준이 파인애플 주스를 샀을 때, 서연은 바나나 주스를 사고, 지아는 딸기 주스를 사는 것이 모든 조건을 만족합니다.

따라서, 최종적으로:
- 지아: 딸기 주스
- 민준: 파인애플 주스
- 서연: 바나나 주스
------------------------------


In [ ]:
print("--- 2. Zero-Shot CoT 프롬프트 방식 ---")
cot_prompt = f"""
다음 논리 문제를 풀어주세요:

문제:
{logical_problem}

단계별로 생각해서 각 단서를 어떻게 적용했는지 설명하고, 최종 답을 다음 형식으로 제시해주세요.
지아: [과일 이름]
민준: [과일 이름]
서연: [과일 이름]
"""
cot_messages = [{"role": "user", "content": cot_prompt}]
result_cot = call_openai_api(cot_messages, temperature=0)
print("\n[Zero-Shot CoT 프롬프팅 결과]")
print(result_cot)
print("-" * 30)

--- 2. Zero-Shot CoT 프롬프트 방식 ---

[Zero-Shot CoT 프롬프팅 결과]
이 문제를 해결하기 위해 주어진 단서를 단계별로 분석해 보겠습니다.

1. **단서 1**: 서연이 산 주스에 사용한 과일의 한국어 이름은 민준이 산 과일 글자보다 크다.
   - 과일 이름의 글자 수를 비교합니다.
   - 파인애플(5글자), 바나나(3글자), 딸기(2글자)
   - 따라서, 서연이 산 주스는 파인애플 주스일 가능성이 높습니다.

2. **단서 2**: 지아가 산 주스의 과일은 민준이 산 주스의 과일보다 일반적으로 크기가 작다.
   - 일반적인 과일 크기를 비교합니다.
   - 딸기 < 바나나 < 파인애플
   - 따라서, 지아가 산 주스는 딸기 주스일 가능성이 높습니다.

3. **단서 3**: 민준이 산 주스에 사용한 과일의 영어 이름에서 알파벳 모음 총 개수는 2보다 크다.
   - 영어 이름의 모음 개수를 세어봅니다.
   - Pineapple (4개: i, e, a, e), Banana (3개: a, a, a), Strawberry (2개: a, e)
   - 따라서, 민준이 산 주스는 바나나 주스일 가능성이 높습니다.

이제 각 단서를 종합하여 결론을 내리겠습니다.

- 서연: 파인애플 주스
- 민준: 바나나 주스
- 지아: 딸기 주스

최종 답:
지아: 딸기
민준: 바나나
서연: 파인애플
------------------------------
